In [1]:
import pandas as pd
import json

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, f1_score
import transformers
from transformers import BertForTokenClassification, AdamW, BertModel
from seqeval.metrics import f1_score, accuracy_score

from tqdm import tqdm, trange
import numpy as np

import glob
import random

from pdf2image import convert_from_path
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from pathlib import Path
import json
import string
import numpy as np
import ast

/data/mmortgage/NERAPI/NER/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-14 08:44:22.795446: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-14 08:44:22.973592: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-14 08:44:23.686890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object fi

In [2]:
entities = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'PAD']

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [4]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(entities),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [5]:
model.load_state_dict(torch.load("/mnt/BERT_NER/BERT_epoch10.model", map_location=torch.device('cuda')))
model.cuda()
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
pdf_dir = r"/data/mmortgage/amal_workspace/BankStatement_Failed/pdf_ld"
ocr_dir = r"/data/mmortgage/amal_workspace/BankStatement_Failed/ocr_ld"
img_dir = r"/data/mmortgage/amal_workspace/BankStatement_Failed/images"
text_dir = r"/data/mmortgage/amal_workspace/BankStatement_Failed/text"
image_bb = "/mnt/NER/bb_images/11_10_22_1"

In [ ]:
for pdf in Path(pdf_dir).glob('**/*.pdf'):
# for i, row in df.iterrows():
#     pdf = f"{row[0]}.pdf"
    out = Path(img_dir)/Path(pdf).stem
    if not out.exists():
        out.mkdir(parents=True, exist_ok=True)
    
    convert_from_path(Path(pdf_dir)/Path(pdf), fmt="jpg",
                     output_folder=out,
                     output_file="")

In [ ]:
def imshow(image):
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()

In [ ]:
def pixel_to_inch(aref, bref, value):
    """

    :param aref: reference in inches
    :param bref: reference in pixel
    :param value: in pixel
    :return:
    """
    return round((aref * value) / bref, 4)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0] ) * (boxB[3] - boxB[1]))
    
    smallerBB_area = boxB if boxAArea>boxBArea else boxA
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea) #+ 1e-6
    # return the intersection over union value
    return iou, smallerBB_area

In [ ]:
def gen_text_ttb(adr, page_no):
    zeroes = "0" * (len(str(len(list((Path(img_dir)/Path(adr)).glob('**/*.jpg')))))-1)
    image_path = str(Path(img_dir)/Path(adr)/Path(f"0001-{zeroes}{page_no}.jpg"))
    print(image_path)
    ocr_path = str(Path(ocr_dir)/Path(f"{adr}.txt"))
    image = cv2.imread(image_path)
#     print(image)
    height_px, width_px, _ = image.shape
    
    gry_img =  cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    smoothing = cv2.bilateralFilter(gry_img, 15, 75, 75)
    kernel_sharpen = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = cv2.filter2D(smoothing, -1, kernel_sharpen)
    
    bw = cv2.threshold(sharpen, 160, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    
    kernel_dil = np.ones((3,3), np.uint8)
    img_dilation = cv2.dilate(bw, kernel_dil, iterations=10)
    
    img_edges = cv2.Canny(img_dilation, 0, 10, apertureSize=3, L2gradient = True)
    
    contours, hierarchy = cv2.findContours(img_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
#     imContour = image.copy()
#     imContour = cv2.drawContours(imContour, contours, -1, (0, 255, 0), 2)
    
    boxes=[]

#     imRect1 = image.copy()
    for i,cnt in enumerate(contours): 
        (x, y, w, h) = cv2.boundingRect(cnt)
        #if w>100 and w<300 and h>50 and h<150:
        boxes.append([x, y, x+w, y+h])
#         cv2.rectangle(imRect1, (x,y), (x+w,y+h), (255, 0, 0), 2)
        
    n = len(boxes)
    l_iou = []
    final_boxes = boxes.copy()
    for i in range(n):
        for j in range(i+1,n):
            iou, rem = bb_intersection_over_union(boxes[i], boxes[j])
    #         iou = round(iou,3)
            l_iou.append(iou)
            if iou>0:
                if rem in final_boxes:
                    final_boxes.remove(rem)
                    
    imRect = image.copy()
    for x1,y1,x2,y2 in final_boxes: 
        cv2.rectangle(imRect, (x1,y1), (x2,y2), (255, 0, 0), 2)
        
    filename = str(Path(image_bb)/ Path(Path(image_path).parent.stem+"_"+Path(image_path).stem))+".jpg"
    cv2.imwrite(filename, imRect)
    
    sorted_cords = sorted(final_boxes , key=lambda k: [k[1], k[0]])
    
    with open(ocr_path, "r") as f:
        ocr = json.load(f)

    page_text = []
    for page in ocr["analyzeResult"]["readResults"]:
        if page["page"]==page_no:
    #     l = page["lines"]  # boundingBox
    #     print(page["page"])
            ocr_cords_sort = sorted(page["lines"] , key=lambda k: [k["boundingBox"][1], k["boundingBox"][0]])
#             print("before", len(ocr_cords_sort))

            height_inch, width_inch = page["height"], page["width"]
            text_lst = []
            for bb in sorted_cords:
                rem_index = []
                for i,v in enumerate(ocr_cords_sort):
                    bb_tl_x = pixel_to_inch(width_inch, width_px, bb[0])
                    bb_tl_y = pixel_to_inch(height_inch, height_px, bb[1])
                    bb_br_x = pixel_to_inch(width_inch, width_px, bb[2])
                    bb_br_y = pixel_to_inch(height_inch, height_px, bb[3])
                    if bb_tl_x<=v["boundingBox"][0]<=bb_br_x and bb_tl_y<=v["boundingBox"][1]<=bb_br_y:
                        text_lst.append(v["text"])
                        rem_index.append(i)
        #                 break
        #         print(rem_index)
                if rem_index:            
                    for index in sorted(rem_index, reverse=True):
                        del ocr_cords_sort[index]

            #         break
#             print("after", len(ocr_cords_sort))
            if ocr_cords_sort:
                for t in ocr_cords_sort:
                    text_lst.append(t["text"])

#             break  
            return " ".join(text_lst)

In [ ]:
# p = Path(img_dir)
# for adr in [f for f in p.iterdir() if f.is_dir()]:
#     page_no = 1
#     text = gen_text_ttb(adr.stem, page_no)
#     with open(f'{text_dir}/{adr.stem}_{page_no}.txt', 'w') as f:
#         f.write(text)

In [ ]:
adr = 
page_no = 1
test_sentence = gen_text_ttb(adr.stem, page_no)

In [6]:
test_sentence = """
BANK OF AMERICA Preferred Rewards BANK OF AMERICA P.O. Box 15284 Wilmington, DE 19850 Customer service information 1.888.888.RWDS (1.888.888.7937) En EspaÃ±ol: 1.800.688.6086 ERIK M LITTLE 4253 CORINTH CHURCH RD LAKE PARK, GA 31636-2923 bankofamerica.com Bank of America, N.A. P.O. Box 25118 Tampa, FL 33622-5118 Your Regular Checking Preferred Rewards Platinum for July 16, 2022 to August 17, 2022 Account number: 3340 2555 6556 ERIK M LITTLE ATM and debit card subtractions -0.00 Other subtractions -1,470.00 Checks -0.00 Service fees -0.00 $126.78 Ending balance on August 17, 2022 Do you follow us on social media? Connect with us on Facebook and Twitter for timely information and to learn more about how to reach your financial goals. f When you use the QRC feature certain information is collected from your mobile device for business purposes. SSM-05-22-0100ÄŒ | 4718133 Page 1 of 4 CYCLE: 10 SPEC: E IMAGE: I BC: GA PULL: B DELIVERY: E TYPE: Account summary $392.14 Beginning balance on July 16, 2022 Deposits and other additions 1,204.64
"""

In [7]:
tokenized_sentence = tokenizer.encode(test_sentence, add_special_tokens=False)
input_ids = torch.tensor([tokenized_sentence[:300]]).cuda()

In [8]:
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [9]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

NameError: name 'tag_values' is not defined

In [ ]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

In [ ]:
d = {"PER":[], "ORG": [], "LOC":[]}
for i, (token, label) in enumerate(zip(new_tokens, new_labels)):
    if label == "B-ORG":
        e = token
        i_n = i + 1
        while new_labels[i_n] == "I-ORG":
            e = e+" "+new_tokens[i_n]
            i_n += 1
        
        if new_labels[i+1]!="I-ORG":
            continue
        d["ORG"].append(e)
        e = ""
        continue
        
    if label == "B-PER":
        e = token
        i_n = i + 1
        while new_labels[i_n] == "I-PER":
            e = e+" "+new_tokens[i_n]
            i_n += 1
        
        if new_labels[i+1]!="I-PER":
            continue
        d["PER"].append(e)
        e = ""
        continue
    
    if label == "B-LOC":
        e = token
        i_n = i + 1
        while new_labels[i_n] == "I-LOC":
            e = e+" "+new_tokens[i_n]
            i_n += 1
        
        if new_labels[i+1]!="I-LOC":
            continue
        d["LOC"].append(e)
        e = ""
        continue   
        

In [ ]:
d